# Some more SciPy muscle flexing: Numerical integration Part 1

My post of last week on drawing Mohr Coulomb's failure criterion based on CU test data got some great response from the community. So I decided to follow this first article up with a couple more. Over the coming period, I will try to post regularly on how to solve geotechnical problems with Python. In my opinion, showing concrete examples of how engineering problems are solved with Python, is the best way to get engineers on board with this type of problem solving. Combining available libraries like SciPy with our subject matter expertise leads to efficiency improvements and greater transparency in workflows.

Today, we will look at numerical integration with SciPy. The ``integrate`` module of SciPy (https://docs.scipy.org/doc/scipy/tutorial/integrate.html) puts fantastic functionality at our disposal. I will split the article into two parts, in this weeks contribution, I will discuss the bearing capacity of a shallow foundation on cohesive soil. The next part will be on integration unit skin friction to obtain pile shaft resistance.

## Problem statement

When a perfectly rigid shallow foundation is loaded vertically, we can postulate a semi-circular failure mechanism in the soil. By taking moments around point $A$ and equating the moment due to the driving force $ P $ to the resisting moments generated by the shear stresses on the failure surface, we can find the collapse load $ P_u$. Note that the normal stresses do not generate any moment contribution. This can be written as an integral expression:

$$ P_u \cdot \frac{B}{2} = \int_{\theta = 0}^{\pi} B \cdot \tau \cdot B \cdot d \theta$$

For a soil profile with uniform undrained shear strength $ S_u $, the solution is trivial as this integral can readily be solved and the following expression is obtained:

$$ P_u = 6.28 \cdot B \cdot S_u $$

<center>
<img src="Images/mechanism_semicircle.png" width=300>
<br><b>Figure 1: Semi-circular failure mechanism (Budhu, 2011)</b></center>



In the example above, we are postulating a semi-circular failure mechanism but this might not be true in reality. As outlined by Budhu (2011), we could choose another failure mechanism by varying the position of the point of rotation from the position shown in Figure 1 to positions above the ground surface (Figure 2).

<center>
<img src="Images/mechanism_arc.png" width=300>
<br><b>Figure 2: Arc failure mechanism (Budhu, 2011)</b></center>

The shear stresses now need to be integrated over the arc, so the integration limits change. The angle $ \theta_0 $ between the vertical and the radius $ R $ indicated in Figure 2 can be calculated as follows:

$$ \theta_0 = \arctan{ \left( \frac{B}{\| OA \|} \right)} $$

The integral expression then becomes:

$$ P_u \cdot \frac{B}{2} = \int_{\theta = -\theta_0}^{\theta_0} R \cdot \tau \cdot R \cdot d \theta$$

The solution is still trivial:

$$ P_u \cdot \frac{B}{2} = 2 \cdot S_u \cdot R^2 \cdot \theta_0 $$

Budhu (2011) shows that the position of the rotation point can be optimised to lead to a minimim value of $ P_u $:

$$ P_{u,min} = 5.52 \cdot B \cdot S_u $$

This is 9% higher than the well-known expression for Prandtl's failure mechanism:

$$ P_u = (2+ \pi )\cdot B \cdot S_u $$

This shows that with a very simple failure mechanism, we can already approach the exact solution quite well.

In this article, we will continue to work with the semi-circular and arc failure mechanisms to illustrate the principles of numerical integration with SciPy.

## Numerical integration with scipy.integrate.quad

Numerical integration with the ``quad`` function from SciPy's integrate module (https://docs.scipy.org/doc/scipy/tutorial/integrate.html#general-integration-quad) requires the definition of an integrand and integration boundaries.

$$ I = \int_{a}^{b} \underbrace{f(x)}_{\text{integrand}} dx $$

We thus need to define $ f(x)$ as a Python function and set the integration limits.

### Semi-circular failure mechanism

For the case of the semi-circular failure mechanism, this is straightforward. Note that we integrate for $ \theta $ so it needs the be the first argument. Other arguments for the integrand can be specified as ``args``. We will also import numpy as we need it.

In [ ]:
import numpy as np
from scipy.integrate import quad

In [ ]:
def integrand_semicircular(theta, undrained_shearstrength, width):
    return width * undrained_shearstrength * width

Peforming the integration is just a matter of calling the ``quad`` function with the appropriate arguments (see function documentation). This function is based on the library QUADPACK written by Robert Piessens, the professor who taught the calculus courses in my bachelor program.

In this example, we will use a foundation width of 1m and an undrained shear strength of 10kPa.

In [ ]:
B = 1 # m
Su = 10 # kPa

In [ ]:
quad(integrand_semicircular, a=0, b=np.pi, args=(Su, B))

We can see that the output is a tuple with the calculated integral and its precision. To only obtain the integrated value, we can simply select the first value using indexing.

In [ ]:
resisting_moments_semicircular = quad(integrand_semicircular, a=0, b=np.pi, args=(Su, B))[0]
Pu_semicircular = 2 * resisting_moments_semicircular / B
Pu_semicircular

We can compare this to the formula proposed by Budhu (2011) and see that both solutions match to the first decimal.

In [ ]:
6.28 * B * Su

### Arc failure mechanism

For the arc failure mechanism, we can use the same solution strategy. First we calculate the angle for the integration limits and the value of the radius (assuming that the rotation point lies above the point A). Let's first assume that the rotation point lies 0.5m above point A.

In [ ]:
OA = 0.5 # m
theta_0 = np.arctan(B / OA) # rad
R = np.sqrt(B ** 2 + OA ** 2) # m

In [ ]:
def integrand_arc(theta, radius, undrained_shearstrength):
    return radius * undrained_shearstrength * radius

In [ ]:
resisting_moments_arc = quad(integrand_arc, a=-theta_0, b=theta_0, args=(R, Su))[0]
Pu_arc = 2 * resisting_moments_arc / B
Pu_arc

We can again see that the values match with the analytically derived formula.

In [ ]:
2 * 2 * Su * (R ** 2) * theta_0 / B

Now, we need to optimise the position of the rotation point. We can first do this by calculating the ultimate load for various positions of the offset to show that a minimum is indeed reached:

In [ ]:
rotationpoint_offsets = np.linspace(0, 1.5 * B, 100)
Pus = np.zeros(100)
theory = np.zeros(100)

In [ ]:
for i, offset in enumerate(rotationpoint_offsets):
    _theta_0 = np.arctan(B / offset)
    _R = np.sqrt(B ** 2 + offset ** 2)
    _resisting_moments_arc = quad(integrand_arc, a=-_theta_0, b=_theta_0, args=(_R, Su))[0]
    Pus[i] = 2 * _resisting_moments_arc / B
    angle = 0.5 * np.pi - _theta_0
    theory[i] = 2 * (2 * Su * _R ** 2 * _theta_0) / B

We can visualise the optimum with Plotly:

In [ ]:
from plotly import subplots
import plotly.graph_objs as go
import plotly.io as pio
pio.templates.default = 'plotly_white'

In [ ]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False)
_data = go.Scatter(x=rotationpoint_offsets, y=Pus, showlegend=True, mode='lines',name='Integration')
fig.append_trace(_data, 1, 1)
_data = go.Scatter(x=rotationpoint_offsets, y=theory, showlegend=True, mode='lines',name='Theory', line=dict(dash='dot'))
fig.append_trace(_data, 1, 1)
fig['layout']['xaxis1'].update(title=r'Offset [m]')
fig['layout']['yaxis1'].update(title=r'$ P_u \ \text{[kN/m]}$')
fig['layout'].update(title='Optimal offset', height=500, width=900)
fig.show()

We can see that there is an optimum and this indeed corresponds with the formula given by Budhu (2011).

In [ ]:
5.52 * B * Su

Now, we should remember that we can also optimise with SciPy. We can even optimise an equation with an integral in it! We just need to write the integrand function inside the objective function. The objective function then minimizes $ P_u $ and calculates the integral during each function evaluation.

In [ ]:
def objective_function(offset, width, su):
    
    def integrand_arc(theta, radius, undrained_shearstrength):
        return radius * undrained_shearstrength * radius
    
    _theta_0 = np.arctan(width / offset)
    _R = np.sqrt(width ** 2 + offset ** 2)
    
    _resisting_moments_arc = quad(integrand_arc, a=-_theta_0, b=_theta_0, args=(_R, su))[0]
    return 2 * _resisting_moments_arc / width

We can perform the optimisation with the minimize function. We search for an optimum between 0 and one foundation width abvove the foundation.

In [ ]:
from scipy.optimize import minimize

In [ ]:
minimize(objective_function, x0=0.5 * B, args=(B, Su), method='SLSQP', bounds=((0, 1.5 * B),))

The optimisation routine finds the optimum with just 9 function evaluations!

## More complex strength profiles

When the strength profiles become more complex, analytical expressions may become difficult to calculate. If the undrained shear strength depends on depth and layer transitions are involved, the integral will have to be calculated in multiple steps. When a data-driven strength profile (e.g. derived from CPT) is used, this is highly unpractical. Not so with the numerical integration. We can just build in whatever model for undrained shear strenght in our integrand and SciPy will still work its magic.

Lets assume a bi-linear undrained shear strength profile where the undrained shear strength increases from 3kPa to 8kPa is the top 0.5m and then jumps to 10kPa at the top of the layer below. The increase in this layer is 1kPa/m.

In [ ]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(x=[3, 8, 10, 13], y=[0, 0.5, 0.5, 3], showlegend=False, mode='lines',name='Su')
fig.append_trace(trace1, 1, 1)
fig['layout']['xaxis1'].update(title=r'$ S_u \ \text{[kPa]}$', side='top', anchor='y', range=(0, 15))
fig['layout']['yaxis1'].update(title=r'$ z \ \text{[m]}$', autorange='reversed')
fig['layout'].update(title='Strength profile', height=600, width=400)
fig.show()

The integrand for the arc mechanism then becomes:

In [ ]:
def undrained_shearstrength_function(z):
    if z < 0:
        return 0
    elif 0 <= z <= 0.5:
        return 3 + 10 * z
    else:
        return 10 + 1 * z
    
def integrand_arc_modified(theta, radius, su_func, offset):
    _z = (radius * np.cos(theta) - offset)
    return radius * su_func(_z) * radius

In [ ]:
OA = 0.5 # m
theta_0 = np.arctan(B / OA) # rad
R = np.sqrt(B ** 2 + OA ** 2) # m

In [ ]:
resisting_moments_arc_modified = quad(integrand_arc_modified, a=-theta_0, b=theta_0, args=(R, undrained_shearstrength_function, OA))[0]
Pu_arc = 2 * resisting_moments_arc_modified / B
Pu_arc

In [ ]:
Pus_modified = np.zeros(100)

for i, offset in enumerate(rotationpoint_offsets):
    _theta_0 = np.arctan(B / offset)
    _R = np.sqrt(B ** 2 + offset ** 2)
    _resisting_moments_arc_modified = quad(integrand_arc_modified, a=-_theta_0, b=_theta_0, args=(_R, undrained_shearstrength_function, offset))[0]
    Pus_modified[i] = 2 * _resisting_moments_arc_modified / B

In [ ]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False)
_data = go.Scatter(x=rotationpoint_offsets, y=Pus, showlegend=True, mode='lines',name='Integration')
fig.append_trace(_data, 1, 1)
_data = go.Scatter(x=rotationpoint_offsets, y=theory, showlegend=True, mode='lines',name='Theory', line=dict(dash='dot'))
fig.append_trace(_data, 1, 1)
_data = go.Scatter(x=rotationpoint_offsets, y=Pus_modified, showlegend=True, mode='lines',name='Modified Su', line=dict(dash='dot'))
fig.append_trace(_data, 1, 1)
fig['layout']['xaxis1'].update(title=r'Offset [m]')
fig['layout']['yaxis1'].update(title=r'$ P_u \ \text{[kN/m]}$')
fig['layout'].update(title='Optimal offset', height=500, width=900)
fig.show()

In [ ]:
def objective_function_modified(offset, width, su):
    
    def integrand_arc_modified(theta, radius, su_func, _offset):
        _z = (radius * np.cos(theta) - _offset)
        return radius * su_func(_z) * radius
    
    _theta_0 = np.arctan(width / offset)
    _R = np.sqrt(width ** 2 + offset ** 2)
    
    _resisting_moments_arc = quad(integrand_arc_modified, a=-_theta_0, b=_theta_0, args=(_R, su, offset))[0]
    return 2 * _resisting_moments_arc / width

In [ ]:
minimize(objective_function_modified, x0=0.5 * B, args=(B, undrained_shearstrength_function), method='SLSQP', bounds=((0, 1.5 * B),))

## Double integration

When a foundation with finite out-of-plane dimensions is considered, we can take into account that there is also a resistance component generated by side shear. The shearing resistance on the entire dark yellow area needs to be calculated and moment contributions of each of these elementary shearing resistances need to be taken into account.

<center>
<img src="Images/mechanism_arc_sideshear.png" width=300>
<br><b>Figure 3: Side shear on arc failure mechanism (colored in dark yellow)</b></center>

Such an elementary moment can be written as:

$$ dM = S_u \cdot dA \cdot r = S_u \cdot dr \cdot r \cdot d \theta \cdot r $$

We need to set appropriate integration limits as $ r $ will vary depending on the angle considered. When $ \theta $ varies from $-\theta_0$ to $\theta_0$, $r$ will vary from $ \frac{\| OA \|}{\cos \theta} $ to $ R$.

So the expression for the side-shear moment becomes:

$$ M_{\text{side shear}} = \int_{\theta=-\theta_0}^{\theta_0} \int_{r=\frac{\| OA \|}{\cos \theta}}^{R} S_u \cdot r^2 \cdot dr \cdot d \theta $$

This may look scary to some but no worries, with SciPy, even double integration becomes a breeze!

The integrand can be written in function of $ \theta $ and $ r$. Since the integration limits on $ r $ depend on $ \theta $, we will use $ r$ as the second argument.

In [ ]:
def integrand_arc_sideshear(theta, r, undrained_shearstrength):
    return undrained_shearstrength * r ** 2

The SciPy ``integrate`` module has the ``dblquad`` function for double integration:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.dblquad.html#scipy.integrate.dblquad

To call the ``dblquad`` function, we need to specify the lower and upper limit of the first variable ($ \theta $ in this case). The integration limits of the second variable can be written as lambda functions. To learn more about lambda functions, use this blog post by Real Python: https://realpython.com/python-lambda/.

In [ ]:
from scipy.integrate import dblquad

We can carry out the integration for a rotation point half a meter above the ground surface.

In [ ]:
OA = 0.5 # m
theta_0 = np.arctan(B / OA) # rad
R = np.sqrt(B ** 2 + OA ** 2) # m
Su = 10 # kPa

In [ ]:
dblquad(integrand_arc_sideshear, a=-theta_0, b=theta_0, gfun=lambda theta: OA / np.cos(theta), hfun=R, args=(Su,))

Note that this moment contribution is given in kNm whereas the previous ones were in kNm per meter out-of-plane length (plane strain conditions). To calculate the total moment for a footing with finite out-of-plane length $ L $, we can multiply the plane strain component by L and take the side shear component twice (at each extremity):

$$ M_{\text{total}} = L \cdot M_{\text{plane strain}} + 2 \cdot M_{\text{side shear}} $$

We can then calculate $ P_u$ for such a 3D case:

$$ P_{u,\text{3D}} = \frac{M_{\text{total}}}{B / 2} $$

## Conclusion

SciPy's integrate module is another very useful addition to the toolset of the Pythonic geotechnical engineer. By formulating the a problem as an integral problem, numerical integration can be applied, even to complex integrands. Functions using numerical integration can also easily be integrated in other workflows (e.g. optimisation routines).

Let me know what you think of this demo or share geotechnical integration problems which you would like to solve with Python.